In [3]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 325.1 kB/s eta 0:06:25
   ---------------------------------------- 0.1/124.9 MB 465.5 kB/s eta 0:04:29
   ---------------------------------------- 0.2/124.9 MB 1.1 MB/s eta 0:01:51
   ---------------------------------------- 0.5/124.9 MB 2.0 MB/s eta 0:01:02
   ---------------------------------------- 1.0/124.9 MB 3.7 MB/s eta 0:00:34
    --------------------------------------- 1.6/124.9 MB 5.0 MB/s eta 0:00:25
    --------------------------------------- 2.1/124.9 MB 5.9 MB/s eta 0:00:21
    --------------------------------------- 2.7/124.9 MB 6.6 MB/s eta 0:00:19
   - -------------------------------------- 3.3/124.9 MB 7.2 MB/s eta 0:00:17
   - -------------------------------------- 3.8/124.9 MB 7.7 MB/s eta 0:00:16
   

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [5]:
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.956043956043956,
  'support': 270},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.915995115995116,
  'support': 300}}

In [7]:
!pip install mlflow

   ---------------------------------------- 0.0/26.3 MB ? eta -:--:--
   ---------------------------------------- 0.2/26.3 MB 3.5 MB/s eta 0:00:08
    --------------------------------------- 0.5/26.3 MB 4.7 MB/s eta 0:00:06
   - -------------------------------------- 1.0/26.3 MB 6.9 MB/s eta 0:00:04
   -- ------------------------------------- 1.5/26.3 MB 8.1 MB/s eta 0:00:04
   --- ------------------------------------ 2.1/26.3 MB 8.9 MB/s eta 0:00:03
   ---- ----------------------------------- 2.7/26.3 MB 9.4 MB/s eta 0:00:03
   ---- ----------------------------------- 3.2/26.3 MB 9.8 MB/s eta 0:00:03
   ----- ---------------------------------- 3.8/26.3 MB 10.0 MB/s eta 0:00:03
   ------ --------------------------------- 4.3/26.3 MB 10.2 MB/s eta 0:00:03
   ------- -------------------------------- 4.9/26.3 MB 10.4 MB/s eta 0:00:03
   -------- ------------------------------- 5.4/26.3 MB 10.5 MB/s eta 0:00:02
   --------- ------------------------------ 6.0/26.3 MB 10.6 MB/s eta 0:00:02
 

In [8]:
import mlflow

In [10]:
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [14]:
mlflow.set_experiment("Basic Experiment")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")

with mlflow.start_run(run_name="Logistic Regression Run"):
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recall_class_0': report_dict['0']['recall'],
        'recall_class_1': report_dict['1']['recall'],
        'f1_score_macro': report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr, "Logistic Regression")  

2024/08/11 23:12:24 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/11 23:12:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression Run at: http://127.0.0.1:5000/#/experiments/451293622317595094/runs/546f74fb54704b41bd4ef7b772d6c9e9.
2024/08/11 23:12:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/451293622317595094.
